## **Desafio de Classificação:**

Utilizando a metodologia básica de um projeto para ciência de dados, implemente possíveis soluções para o seguinte estudo de caso (https://www.kaggle.com/datasets/ahsan81/hotel-reservations-classification-dataset?resource=download). Desenvolva e implemente métodos que se propõem a responder a questão: **Será que um hóspede irá cancelar sua reserva?**

Em seu projeto deverá constar uma análise detalhada do dataset e descrição das transformações de dados realizadas com suas respectivas justificativas. Adicionalmente, crie um baseline com as técnicas contidas nesta aula. Como forma de superar este baseline, pesquise por métodos e técnicas mais complexas. Você deverá entregar um relatório com os seguintes itens:

- Descrição do problema e análise dos dados
- Descrição das técnicas utilizadas
- Interpretação dos resultados obtidos
- Conclusão
- Apêndice (descrição das técnicas de Classificação apresentadas pelos colegas durante os seminários)

Adicionalmente, no relatório coloque a url de seu repositório on-line para consulta. Seu código deve estar comentado!!

Sistema de Avaliação:

- (1,0) Qualidade do relatório
- (1,0) Análise dos dados e descrição do problema
- (3,0) Implementação da solução
- (3,0) Interpretação dos resultados
- (2,0) Resumo das técnicas dos seminários de Classificação


# Cancel Culture

This is a project intended to predict whether or not a client will cancel a reservation. The data comes from a Kaggle dataset, and we will use many common ML algorithms and make comparison between them.

### Table of Contents

- [Introduction](##Introduction)
- [Exploratory Data Analysis](##Exploratory-Data-Analysis)


## Introduction

{this is a project that looks to predict whether or not a client will cancel a reservation}

{we use the kaggle.com/blah dataset}

{we do: exploratory data analysis}

{model training: knn, svm...}

{data interpretation and conclusion}


## Exploratory Data Analysis & Pre-processing


In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import (
    StratifiedKFold,
    cross_val_score,
    cross_val_predict,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier

import time

from imblearn.over_sampling import RandomOverSampler

In [ ]:
# Load dataset
df = pd.read_csv("./hotel_reservations.csv")
df.head()

Lets get started with the analysis!

First, let's check our attributes in a more readable way:


In [ ]:
df.info()

Some of these features are categorical (`<object>`), so let's check what these features have!


In [ ]:
n_top_values = 5
for col in df.select_dtypes(include="object"):
    unique_values = df[col].nunique()
    top_values = sorted(df[col].unique())[:n_top_values]
    top_values_str = ", ".join([f"<{v}>" for v in top_values])
    print(
        f"{col}: ({unique_values} distinct) {top_values_str + ', ...' if len(top_values) == n_top_values else top_values_str}"
    )

The `Booking_ID` is meaningless for our algorithms, so we can remove that column.

Also, we can convert our categorical features to numerical by using some encoding. There is some methods available, such as one-hot encoding, but we're going to go with label encoding because it's simpler.


In [ ]:
df.drop("Booking_ID", axis=1, inplace=True)

# identify categorical columns
cat_cols = df.select_dtypes(include="object").columns

# label encode categorical columns
for col in cat_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

## Model Training


### Splitting the dataset


In [ ]:
X = df.drop("booking_status", axis=1)
Y = df["booking_status"]

ros = RandomOverSampler(random_state=0)
X, Y = ros.fit_resample(X, Y)

n_folds = 10
kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=0)

### Creating auxiliary functions


In [ ]:
# Evaluate models by using cross validation
def evaluate_model(model, model_name, frac=None):
    start_time = time.time()

    # This is need in order to SVM to run in a reasonable amount of time
    if frac is not None:
        X_sample, Y_sample = X.sample(frac=frac), Y.sample(frac=frac)
    else:
        X_sample, Y_sample = X, Y

    scores = cross_val_score(model, X_sample, Y_sample, cv=kf, n_jobs=-1)
    end_time = time.time()

    print(
        "(%0.1fs) Accuracy with %s: %0.4f ± %0.4f"
        % (end_time - start_time, model_name, scores.mean(), scores.std())
    )

### KNN


In [ ]:
for i in range(1, 4):
    knn = KNeighborsClassifier(n_neighbors=i)
    evaluate_model(knn, f"{i}-KNN")

### Naive Bayes


In [ ]:
nb = GaussianNB()
evaluate_model(nb, "Naive Bayes")

### Logistic Regression


In [ ]:
rlog = LogisticRegression(max_iter=5000)
evaluate_model(rlog, "Logistic Regression")

### SVM


In [ ]:
svm = SVC(kernel="linear")
evaluate_model(svm, "SVM Linear", frac=0.1)

svm_rbf = SVC(kernel="rbf")
evaluate_model(svm, "SVM RBF", frac=0.1)

svm_poly = SVC(kernel="poly", degree=3)
evaluate_model(svm, "SVM Poly", frac=0.1)

### Decision Tree


In [ ]:
dct = tree.DecisionTreeClassifier()
evaluate_model(dct, "Decision Tree")

dct_entropy = tree.DecisionTreeClassifier(criterion="entropy")
evaluate_model(dct_entropy, "Decision Tree (criterion=entropy)")

dct_larger_depth = tree.DecisionTreeClassifier(max_depth=15)
evaluate_model(dct, "Decision Tree (max_depth=15)")

### Random Forest


In [ ]:
rf = RandomForestClassifier(n_estimators=300, max_depth=15)
evaluate_model(rf, "Random Forest")

### XGBoost


In [ ]:
xgb = XGBClassifier()
evaluate_model(xgb, "XGBoost")